# Predicting the Sentiment of Movie Reviews

There are two goals for this analysis. The first is to accurately predict the sentiment of movie reviews, and the second is to develop my model in such a way that its outputs can be analyzed with TensorBoard. This is the first time that I am using TensorBoard, so I want to have a somewhat challenging task, and not use a huge dataset. There are 25,000 training and testing reviews, so this model can train multiple iterations overnight on my MacBook Pro. The data is provided by a Kaggle competition from 2015 (https://www.kaggle.com/c/word2vec-nlp-tutorial). Despite it having concluded, it can still be used as an excellent learning opportunity. The sections of this analysis are:
- Inspect the Data
- Clean and Format the Data
- Build and Train the Model
- Make the Predictions
- Summary

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk, re, time
from nltk.corpus import stopwords
from string import punctuation
from collections import defaultdict
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from collections import namedtuple

Using TensorFlow backend.


In [2]:
# Load the data
train = pd.read_csv("labeledTrainData.tsv", delimiter="\t")
test = pd.read_csv("testData.tsv", delimiter="\t")

# Inspect the Data

In [3]:
train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [4]:
test.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [5]:
print(train.shape)
print(test.shape)

(25000, 3)
(25000, 2)


The reviews are rather long, so we won't be using all of the text to train our model. Using all of the text would increase our training to a longer timeframe than I would rather give to this project, but it should make the predictions more accurate.

In [6]:
# Inspect the reviews
for i in range(3):
    print(train.review[i])
    print()

With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally star

In [7]:
# Check for any null values
print(train.isnull().sum())
print(test.isnull().sum())

id           0
sentiment    0
review       0
dtype: int64
id        0
review    0
dtype: int64


# Clean and Format the Data

In [8]:
def clean_text(text, remove_stopwords=True):
    '''Clean the text, with the option to remove stopwords'''
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"<br />", " ", text)
    text = re.sub(r"[^a-z]", " ", text)
    text = re.sub(r"   ", " ", text) # Remove any extra spaces
    text = re.sub(r"  ", " ", text)
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Return a list of words
    return(text)

Clean the training and testing reviews

In [9]:
train_clean = []
for review in train.review:
    train_clean.append(clean_text(review))

In [10]:
test_clean = []
for review in test.review:
    test_clean.append(clean_text(review))

In [11]:
# Inspect the cleaned reviews
for i in range(3):
    print(train_clean[i])
    print()

stuff going moment mj i ve started listening music watching odd documentary there watched wiz watched moonwalker again maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj s feeling towards press also obvious message drugs bad m kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice him the actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond me mj overheard plans nah joe pesci s character ranted wanted people know supplying drugs etc dunno maybe hates mj s music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad seq

In [13]:
# Tokenize the reviews
all_reviews = train_clean + test_clean
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_reviews)
print("Fitting is complete.")

train_seq = tokenizer.texts_to_sequences(train_clean)
print("train_seq is complete.")

test_seq = tokenizer.texts_to_sequences(test_clean)
print("test_seq is complete")

Fitting is complete.
train_seq is complete.
test_seq is complete


In [14]:
# Find the number of unique tokens
word_index = tokenizer.word_index
print("Words in index: %d" % len(word_index))

Words in index: 99426


In [15]:
# Inspect the reviews after they have been tokenized
for i in range(3):
    print(train_seq[i])
    print()

[445, 86, 489, 10939, 8, 61, 583, 2603, 120, 68, 957, 560, 53, 212, 24485, 212, 17247, 219, 193, 97, 20, 695, 2565, 124, 109, 15, 520, 3954, 193, 27, 246, 654, 2352, 1261, 17247, 90, 4782, 90, 712, 3, 305, 86, 16, 358, 1846, 542, 1219, 3592, 10939, 1, 485, 871, 3538, 23, 526, 673, 1414, 19, 63, 5305, 2089, 1118, 185, 413, 1523, 817, 2583, 7, 10939, 477, 86, 665, 85, 272, 114, 578, 10939, 34480, 29662, 148, 2, 10939, 381, 13, 59, 26, 381, 210, 15, 252, 178, 10, 751, 712, 3, 142, 341, 464, 145, 16427, 4121, 1718, 635, 876, 10547, 1018, 12089, 890, 1067, 1652, 416, 10939, 265, 19, 596, 141, 10939, 18336, 2302, 15821, 876, 10547, 1, 34, 38190, 388, 21, 49, 17539, 1414, 434, 9821, 193, 4238, 10939, 1, 120, 669, 520, 96, 7, 10939, 1555, 444, 2271, 138, 2137, 2383, 635, 23, 72, 117, 4750, 5364, 307, 1326, 31136, 19, 635, 556, 888, 665, 697, 6, 452, 195, 547, 138, 689, 3386, 1234, 790, 56, 1239, 268, 2, 21, 7, 10939, 6, 580, 78, 476, 32, 21, 245, 706, 158, 276, 113, 7674, 673, 3526, 10939, 1, 

In [16]:
# Find the length of reviews
lengths = []
for review in train_seq:
    lengths.append(len(review))

for review in test_seq:
    lengths.append(len(review))

# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])

In [17]:
lengths.counts.describe()

count    50000.000000
mean       131.048860
std         98.409141
min          3.000000
25%         70.000000
50%         98.000000
75%        160.000000
max       1476.000000
Name: counts, dtype: float64

In [18]:
print(np.percentile(lengths.counts, 80))
print(np.percentile(lengths.counts, 85))
print(np.percentile(lengths.counts, 90))
print(np.percentile(lengths.counts, 95))

182.0
213.0
258.0
338.0


I'm going to use 200 as the maximum length of a review. Although longer would be better, I want to limit the training time, and this value will still provide the full text to more than 80% of the reviews.

In [19]:
# Pad and truncate the questions so that they all have the same length.
max_review_length = 200

train_pad = pad_sequences(train_seq, maxlen = max_review_length)
print("train_pad is complete.")

test_pad = pad_sequences(test_seq, maxlen = max_review_length)
print("test_pad is complete.")

train_pad is complete.
test_pad is complete.


In [20]:
# Inspect the reviews after padding has been completed. 
for i in range(3):
    print(train_pad[i,:100])
    print()

[   90   712     3   305    86    16   358  1846   542  1219  3592 10939
     1   485   871  3538    23   526   673  1414    19    63  5305  2089
  1118   185   413  1523   817  2583     7 10939   477    86   665    85
   272   114   578 10939 34480 29662   148     2 10939   381    13    59
    26   381   210    15   252   178    10   751   712     3   142   341
   464   145 16427  4121  1718   635   876 10547  1018 12089   890  1067
  1652   416 10939   265    19   596   141 10939 18336  2302 15821   876
 10547     1    34 38190   388    21    49 17539  1414   434  9821   193
  4238 10939     1   120]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

[  972   762 17403  6324  4734   315  9375  8934  3636  1946  7468  2265
 11236  1537  2500  2101  2943  3032   958  1723  1252  1524 35849  1862
  1606  2500  1406  9610 39014  4034  

In [21]:
# Creating the training and validation sets
x_train, x_valid, y_train, y_valid = train_test_split(train_pad, train.sentiment, test_size = 0.15, random_state = 2)
x_test = test_pad

In [22]:
# Inspect the shape of the data
print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)

(21250, 200)
(3750, 200)
(25000, 200)


# Build and Train the Model

In [23]:
def get_batches(x, y, batch_size):
    '''Create the batches for the training and validation data'''
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [87]:
def get_test_batches(x, batch_size):
    '''Create the batches for the testing data'''
    n_batches = len(x)//batch_size
    x = x[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size]

In [53]:
def build_rnn(n_words, embed_size, batch_size, lstm_size, num_layers, 
              dropout, learning_rate, multiple_fc, fc_units):
    '''Build the Recurrent Neural Network'''

    tf.reset_default_graph()

    # Declare placeholders we'll feed into the graph
    with tf.name_scope('inputs'):
        inputs = tf.placeholder(tf.int32, [None, None], name='inputs')

    with tf.name_scope('labels'):
        labels = tf.placeholder(tf.int32, [None, None], name='labels')

    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    # Create the embeddings
    with tf.name_scope("embeddings"):
        embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
        embed = tf.nn.embedding_lookup(embedding, inputs)

    # Build the RNN layers
    with tf.name_scope("RNN_layers"):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    
    # Set the initial state
    with tf.name_scope("RNN_init_state"):
        initial_state = cell.zero_state(batch_size, tf.float32)

    # Run the data through the RNN layers
    with tf.name_scope("RNN_forward"):
        outputs, final_state = tf.nn.dynamic_rnn(cell, embed,
                                                 initial_state=initial_state)    
    
    # Create the fully connected layers
    with tf.name_scope("fully_connected"):
        
        # Initialize the weights and biases
        weights = tf.truncated_normal_initializer(stddev=0.1)
        biases = tf.zeros_initializer()
        
        dense = tf.contrib.layers.fully_connected(outputs[:, -1],
                                                  num_outputs = fc_units,
                                                  activation_fn = tf.sigmoid,
                                                  weights_initializer = weights,
                                                  biases_initializer = biases)
        dense = tf.contrib.layers.dropout(dense, keep_prob)
        
        # Depending on the iteration, use a second fully connected layer
        if multiple_fc == True:
            dense = tf.contrib.layers.fully_connected(dense,
                                                      num_outputs = fc_units,
                                                      activation_fn = tf.sigmoid,
                                                      weights_initializer = weights,
                                                      biases_initializer = biases)
            dense = tf.contrib.layers.dropout(dense, keep_prob)
    
    # Make the predictions
    with tf.name_scope('predictions'):
        predictions = tf.contrib.layers.fully_connected(dense, 
                                                        num_outputs = 1, 
                                                        activation_fn=tf.sigmoid,
                                                        weights_initializer = weights,
                                                        biases_initializer = biases)
        tf.summary.histogram('predictions', predictions)
    
    # Calculate the cost
    with tf.name_scope('cost'):
        cost = tf.losses.mean_squared_error(labels, predictions)
        tf.summary.scalar('cost', cost)
    
    # Train the model
    with tf.name_scope('train'):    
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

    # Determine the accuracy
    with tf.name_scope("accuracy"):
        correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels)
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
    
    # Merge all of the summaries
    merged = tf.summary.merge_all()    

    # Export the nodes 
    export_nodes = ['inputs', 'labels', 'keep_prob', 'initial_state', 'final_state','accuracy',
                    'predictions', 'cost', 'optimizer', 'merged']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

In [92]:
def train(model, epochs, log_string):
    '''Train the RNN'''

    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        # Used to determine when to stop the training early
        valid_loss_summary = []
        
        # Keep track of which batch iteration is being trained
        iteration = 0

        print()
        print("Training Model: {}".format(log_string))

        train_writer = tf.summary.FileWriter('./logs/3/train/{}'.format(log_string), sess.graph)
        valid_writer = tf.summary.FileWriter('./logs/3/valid/{}'.format(log_string))

        for e in range(epochs):
            state = sess.run(model.initial_state)
            
            # Record progress with each epoch
            train_loss = []
            train_acc = []
            val_acc = []
            val_loss = []

            with tqdm(total=len(x_train)) as pbar:
                for _, (x, y) in enumerate(get_batches(x_train, y_train, batch_size), 1):
                    feed = {model.inputs: x,
                            model.labels: y[:, None],
                            model.keep_prob: dropout,
                            model.initial_state: state}
                    summary, loss, acc, state, _ = sess.run([model.merged, 
                                                             model.cost, 
                                                             model.accuracy, 
                                                             model.final_state, 
                                                             model.optimizer], 
                                                            feed_dict=feed)                
                    
                    # Record the loss and accuracy of each training batch
                    train_loss.append(loss)
                    train_acc.append(acc)
                    
                    # Record the progress of training
                    train_writer.add_summary(summary, iteration)
                    
                    iteration += 1
                    pbar.update(batch_size)
            
            # Average the training loss and accuracy of each epoch
            avg_train_loss = np.mean(train_loss)
            avg_train_acc = np.mean(train_acc) 

            val_state = sess.run(model.initial_state)
            with tqdm(total=len(x_valid)) as pbar:
                for x, y in get_batches(x_valid, y_valid, batch_size):
                    feed = {model.inputs: x,
                            model.labels: y[:, None],
                            model.keep_prob: 1,
                            model.initial_state: val_state}
                    summary, batch_loss, batch_acc, val_state = sess.run([model.merged, 
                                                                          model.cost, 
                                                                          model.accuracy, 
                                                                          model.final_state], 
                                                                         feed_dict=feed)
                    
                    # Record the validation loss and accuracy of each epoch
                    val_loss.append(batch_loss)
                    val_acc.append(batch_acc)
                    pbar.update(batch_size)
            
            # Average the validation loss and accuracy of each epoch
            avg_valid_loss = np.mean(val_loss)    
            avg_valid_acc = np.mean(val_acc)
            valid_loss_summary.append(avg_valid_loss)
            
            # Record the validation data's progress
            valid_writer.add_summary(summary, iteration)

            # Print the progress of each epoch
            print("Epoch: {}/{}".format(e, epochs),
                  "Train Loss: {:.3f}".format(avg_train_loss),
                  "Train Acc: {:.3f}".format(avg_train_acc),
                  "Valid Loss: {:.3f}".format(avg_valid_loss),
                  "Valid Acc: {:.3f}".format(avg_valid_acc))

            # Stop training if the validation loss does not decrease after 3 epochs
            if avg_valid_loss > min(valid_loss_summary):
                print("No Improvement.")
                stop_early += 1
                if stop_early == 3:
                    break   
            
            # Reset stop_early if the validation loss finds a new low
            # Save a checkpoint of the model
            else:
                print("New Record!")
                stop_early = 0
                checkpoint = "/Users/Dave/Desktop/Programming/Personal Projects/Movie_Reviews_Kaggle/sentiment_{}.ckpt".format(log_string)
                saver.save(sess, checkpoint)

In [93]:
# The default parameters of the model
n_words = len(word_index)
embed_size = 300
batch_size = 250
lstm_size = 128
num_layers = 2
dropout = 0.5
learning_rate = 0.001
epochs = 100
multiple_fc = False
fc_units = 256

In [94]:
# Train the model with the desired tuning parameters
for lstm_size in [64,128]:
    for multiple_fc in [True, False]:
        for fc_units in [128, 256]:
            log_string = 'ru={},fcl={},fcu={}'.format(lstm_size,
                                                      multiple_fc,
                                                      fc_units)
            model = build_rnn(n_words = n_words, 
                              embed_size = embed_size,
                              batch_size = batch_size,
                              lstm_size = lstm_size,
                              num_layers = num_layers,
                              dropout = dropout,
                              learning_rate = learning_rate,
                              multiple_fc = multiple_fc,
                              fc_units = fc_units)            
            train(model, epochs, log_string)


Training Model: ru=64,fcl=True,fcu=128


100%|██████████| 3750/3750 [00:11<00:00, 320.47it/s]


Epoch: 0/100 Train Loss: 0.241 Train Acc: 0.581 Valid Loss: 0.180 Valid Acc: 0.744
New Record!


100%|██████████| 3750/3750 [00:13<00:00, 290.49it/s]


Epoch: 1/100 Train Loss: 0.140 Train Acc: 0.817 Valid Loss: 0.133 Valid Acc: 0.821
New Record!


...

Epoch: 4/100 Train Loss: 0.047 Train Acc: 0.946 Valid Loss: 0.118 Valid Acc: 0.857
No Improvement.


  0%|          | 0/21250 [00:00<?, ?it/s]

Epoch: 5/100 Train Loss: 0.043 Train Acc: 0.950 Valid Loss: 0.120 Valid Acc: 0.859
No Improvement.


100%|██████████| 3750/3750 [00:20<00:00, 182.65it/s]

Epoch: 6/100 Train Loss: 0.042 Train Acc: 0.951 Valid Loss: 0.120 Valid Acc: 0.861
No Improvement.


# Make the Predictions

In [132]:
def make_predictions(lstm_size, multiple_fc, fc_units, checkpoint):
    '''Predict the sentiment of the testing data'''
    
    # Record all of the predictions
    all_preds = []

    model = build_rnn(n_words = n_words, 
                      embed_size = embed_size,
                      batch_size = batch_size,
                      lstm_size = lstm_size,
                      num_layers = num_layers,
                      dropout = dropout,
                      learning_rate = learning_rate,
                      multiple_fc = multiple_fc,
                      fc_units = fc_units) 
    
    with tf.Session() as sess:
        saver = tf.train.Saver()
        # Load the model
        saver.restore(sess, checkpoint)
        test_state = sess.run(model.initial_state)
        for _, x in enumerate(get_test_batches(x_test, batch_size), 1):
            feed = {model.inputs: x,
                    model.keep_prob: 1,
                    model.initial_state: test_state}
            predictions = sess.run(model.predictions, feed_dict=feed)
            for pred in predictions:
                all_preds.append(float(pred))
                
    return all_preds

I am going to compare the results of the best three models, based on the validation data. Then average the predictions of these three models, which should produce an even better set of predictions. 

In [139]:
checkpoint1 = "/Users/Dave/Desktop/Programming/Personal Projects/Movie_Reviews_Kaggle/sentiment_ru=128,fcl=False,fcu=256.ckpt"
checkpoint2 = "/Users/Dave/Desktop/Programming/Personal Projects/Movie_Reviews_Kaggle/sentiment_ru=128,fcl=False,fcu=128.ckpt"
checkpoint3 = "/Users/Dave/Desktop/Programming/Personal Projects/Movie_Reviews_Kaggle/sentiment_ru=64,fcl=True,fcu=256.ckpt"

In [140]:
# Make predictions using the best 3 models
predictions1 = make_predictions(128, False, 256, checkpoint1)
predictions2 = make_predictions(128, False, 128, checkpoint2)
predictions3 = make_predictions(64, True, 256, checkpoint3)

In [155]:
# Average the best three predictions
predictions_combined = (pd.DataFrame(predictions1) + pd.DataFrame(predictions2) + pd.DataFrame(predictions3))/3

In [142]:
def write_submission(predictions, string):
    '''write the predictions to a csv file'''
    submission = pd.DataFrame(data={"id":test["id"], "sentiment":predictions})
    submission.to_csv("submission_{}.csv".format(string), index=False, quoting=3)

In [162]:
write_submission(predictions1, "ru=128,fcl=False,fcu=256") 
write_submission(predictions2, "ru=128,fcl=False,fcu=128") 
write_submission(predictions3, "ru=64,fcl=True,fcu=256") 
write_submission(predictions_combined.ix[:,0], "combined") 

The results of the predictions are as follows (Kaggle used area under the ROC curve to evaluation submissions):
- Predictions1: 0.919
- Predictions2: 0.914
- Predictions3: 0.916
- Combined Predictions: 0.935

# Summary

I am rather pleased by how this analysis has finished. Now I am much more confident in using TensorBoard to improve the design of a model and I have achieved rather good results. The combined predictions' submission ranks 206 out of 578, top 35.6%. This result could have been improved by using a larger model, using pretrained vectors (such as GloVe), and using an ensemble of more predictions. Although it would be nice to carry out these efforts and improve my results, I feel that would not be the best use of my time. There are more complicated projects that I would like to work on now, rather than training this model multiple times for a competition that has already concluded. 